# Models_and_stacking
Ноутбук, содержащий историю эволюции итоговой модели.
- Сравнение промежуточных моделей
- Стакинг промежуточных моделей

# Data

In [1]:
import pandas as pd
import numpy as np

from sklearn import neighbors
from catboost import CatBoostRegressor
from sklearn import tree                             

from sklearn.model_selection import train_test_split

In [2]:
%store -r df

preX = pd.concat(
[df[['brand','sell_id','price','engineDisplacement', 'enginePower','mileage','car_age', 'model_product_time']] ,
pd.get_dummies(df.bodyType,prefix='bodyType'),
pd.get_dummies(df.fuelType,prefix='fuelType'),
pd.get_dummies(df.vehicleTransmission,prefix='vehicleTransmission'),
pd.get_dummies(df.Привод,prefix='Привод'),
pd.get_dummies(df.Владельцы,prefix='Владельцы'),
pd.get_dummies(df.Владение,prefix='Владение'),
pd.get_dummies(df.color,prefix='color'),
pd.get_dummies(df.ПТС,prefix='ПТС'),
pd.get_dummies(df.Руль,prefix='Руль'),
pd.get_dummies(df.numberOfDoors,prefix='numberOfDoors'),],axis=1)

preX.sample(5)

test = preX[preX.price==-1]
train = preX[preX.price!=-1]

X=train.drop(['brand','sell_id','price'],axis='columns')
y=train.price

### Метрика
$$\Large MAPE= 100 \% * \frac{1}{n}\sum_{t=1}^{n}\frac{\left | Y_t-\hat{Y_t} \right |}{Y_t}$$

In [3]:
'''Функция вывода MAPE'''
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))*100

# Cравнение Decision Tree, CatBoost и k-nears neighbors 

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X ,
    y, test_size=0.2, shuffle=True, random_state=42)

In [5]:
### k-nears neighbors regression
knn = neighbors.KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train, y_train)
print('MAPE:', mape(y_test, knn.predict(X_test)))

MAPE: 39.96060973627035


Первое предсказание есть!

In [6]:
### CatBoost regression
model = CatBoostRegressor(iterations = 10000,
                          random_seed = 42, silent=True)
model.fit(X_train.values, np.log(y_train.values))

print('MAPE:',mape(y_test, np.exp(model.predict(X_test.values)) ))

MAPE: 15.51307854541574


CatBoost намного лучше knn

In [7]:
### Decision Tree regression
model = tree.DecisionTreeRegressor(max_depth=16, max_features=None)
model.fit(X_train,y_train)

print('MAPE:',(mape(y_test, (model.predict(X_test)))))

MAPE: 19.065013706041178


CatBoost показал себя лучше простого решающего дерева и ближайших соседей, можно сделать первый сабмит. 

Начиная с этого момента метрика будет браться из Kaggle

In [8]:
'''SUBMISSION_1''' ### CatBoost
model = CatBoostRegressor(iterations = 10000,
                          random_seed = 42, silent=True)
model.fit(X.values, np.log(y.values))

price = pd.Series(np.exp(model.predict(test.drop(['brand','sell_id','price'],axis='columns').values)))
price.name = 'price'
price = price.astype('int')
pd.concat([test.sell_id, price],axis=1).to_csv(f'submission_1.csv', index=False)
print('KAGGLE score: 15.81082')

KAGGLE score: 15.81082


# Stacking

## В игру вступает [big_trees](./big_trees.ipynb)
Предсказывает решающими деревьями. Каждое из деревьев обучалось на машинах только одного бренда.

In [9]:
big_pred = pd.read_pickle('big_pred.pickle') # предсказания big_trees

In [10]:
''' SUBMISSION 9 ''' ### big_trees
prediction = pd.Series(big_pred).astype('int')
prediction.name = 'price'


pd.concat([test.sell_id, prediction],axis=1).to_csv(
    'submission_9.csv', index=False)

print('KAGGLE score: 12.86383')

KAGGLE score: 12.86383


Отличный результат!

А как совмещать предсказания? 
- Арифметическим средним. Складываются два или более предсказания, после чего их сумма делится на их же количество

In [11]:
( ((12+7)/2 )+3)/2, (12+7+3)/3 # пример регуляции влияния предсказаний

(6.25, 7.333333333333333)

In [12]:
''' SUBMISSION 2 ''' ### CatBoost & big_trees
pd.concat([test.sell_id, (price + big_pred)/2],axis=1).to_csv(
    f'submission_2.csv', index=False)
print('KAGGLE score: 12.82608')

KAGGLE score: 12.82608


Стэкинг catBoost & big_tree показал улучшение метрики 

## В игру вступает [lil_trees](./lil_trees.ipynb).
Предсказывает решающими деревьями. Каждое дерево обучено на одной модели авто.

In [13]:
lil_pred = pd.read_pickle('lil_pred.pickle') # это его предсказания

In [14]:
# Cтаршие алгоритмы подсказывают младшему авто, моделей которых нет в датасете
for i in lil_pred[lil_pred==0.36787944117144233].index:
    lil_pred.at[i] = (big_pred[i] + price[i])/2

In [15]:
''' SUBMISSION 8 ''' ### lil_trees
prediction = (lil_pred).astype('int')
prediction.name = 'price'


pd.concat([test.sell_id, prediction],axis=1).to_csv(
    'submission_8.csv', index=False)

print('KAGGLE score: 12.29984')

KAGGLE score: 12.29984


Деревья lil_trees показали себя лучше всех в одиночку

In [16]:
''' SUBMISSION 3 ''' ### lil_pred & big_pred & CatBoost
prediction = ((lil_pred + big_pred + price)/3).astype('int')
prediction.name = 'price'


pd.concat([test.sell_id, prediction],axis=1).to_csv(
    'submission_3.csv', index=False)
print('KAGGLE score: 11.58098')

KAGGLE score: 11.58098


Сэкинг всех моделей показал лучший результат из рассмотренных, но, возможно существует лучшая комбинация

In [17]:
'''SUBMISSION 4''' ### lil_pred & big_pred
prediction = ((lil_pred + big_pred)/2).astype('int')
prediction.name = 'price'


pd.concat([test.sell_id, prediction],axis=1).to_csv(
    'submission_4.csv', index=False)

print('KAGGLE score: 11.29318')

KAGGLE score: 11.29318


Лучшая комбинация - деревья lil_pred и big_pred.

Возможно стоит урегулировать влияние моделей друг на друга, это может показать более выскокий результат.

In [18]:
''' SUBMISSION 7 ''' ### 2 lil_pred & big_pred 
prediction = ( ((big_pred + lil_pred )/2 + lil_pred)/2 ).astype('int')
prediction.name = 'price'


pd.concat([test.sell_id, prediction],axis=1).to_csv(
    'submission_7.csv', index=False)

print('KAGGLE score: 11.45419')

KAGGLE score: 11.45419


In [19]:
''' SUBMISSION 5 ''' ### CatBoost & big_pred & 2 lil_pred
prediction = (  ( ((price + big_pred)/2) + lil_pred )/2  ).astype('int')
prediction.name = 'price'


pd.concat([test.sell_id, prediction],axis=1).to_csv(
    'submission_5.csv', index=False)

print('KAGGLE score: 11.36916')

KAGGLE score: 11.36916


In [20]:
''' SUBMISSION 6 ''' ### CatBoost & lil_pred
prediction = ( (price + lil_pred )/2 ).astype('int')
prediction.name = 'price'


pd.concat([test.sell_id, prediction],axis=1).to_csv(
    'submission_6.csv', index=False)

print('KAGGLE score: 12.44002')

KAGGLE score: 12.44002


In [21]:
''' SUBMISSION 10 ''' ### 2 big_pred & lil_pred
prediction = (  (  ( (big_pred + lil_pred)/2) + big_pred)/2  ).astype('int')
prediction.name = 'price'


pd.concat([test.sell_id, prediction],axis=1).to_csv(
    'submission_10.csv', index=False)

print('KAGGLE score: 11.78683')

KAGGLE score: 11.78683


Лучший результат - равный вклад в предсказание от lil_pred и big_pred

Возможно, что лучше всего заполнить пропуски lil_pred именно big_pred'ом, поскольку последний показывает результат лучший, чем CatBoost.

In [22]:
lil_pred = pd.read_pickle('lil_pred.pickle')

for i in lil_pred[lil_pred==0.36787944117144233].index:
    lil_pred.at[i] = big_pred[i]
    
    
''' SUBMISSION 11 ''' ### lil_pred & big_pred
prediction = ((lil_pred + big_pred)/2).astype('int')
prediction.name = 'price'


pd.concat([test.sell_id, prediction],axis=1).to_csv(
    'submission_11.csv', index=False)

print('KAGGLE score: 11.29386')

KAGGLE score: 11.29386


Нет, это маловажно

# Итог
lil_trees показал себя лучше всего, но его работу можно улучшить с помощью big_trees. Это интерпретируется так:

- Появившаяся гиппотеза о том, что для предсказания стоимости авто лучше всего разбираться в его модели - справедлива для имеющихся данных.


- Если плохо знаешь модель авто, но хочешь его оценить - лучше посмотреть на авто, похожее по характеристикам в бренде.

Если спросите, почему я не обучал перспективный CatBoost на разделенных по брендам или моделям данных, то я отвечу, что не успел посмотреть на данную комбинацию.